In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import scikitplot as skplt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.express as px

In [2]:
import seaborn as sns

In [31]:
df = pd.DataFrame([])
sam = pd.DataFrame([],columns=cols)
cols = ['Year','F1_score','F2_score','Precision','Recall','MCC','missed_fire','Warned Fire Count']
for year in range(1999,2020,1):
    Name = 'df_test_' + str(year) + ".parquet.gzip"
    #df = df.append(pq.read_table(source=Name).to_pandas())
    df = pq.read_table(source=Name).to_pandas()
    F1_score = round(f1_score(df['fire_occurred'], df['predicted_fire_risk'], average='binary'),5)
    F2_score = round(fbeta_score(df['fire_occurred'], df['predicted_fire_risk'], average='binary', beta=2),5)
    Precision = "{:.2%}".format(round(precision_score(df['fire_occurred'], df['predicted_fire_risk'], average='binary'),5))
    Recall = "{:.2%}".format(round(recall_score(df['fire_occurred'], df['predicted_fire_risk'], average='binary'),5))
    MCC = round(matthews_corrcoef(df['fire_occurred'], df['predicted_fire_risk']),5)
    
    #Check if we missed predicting any fire reporting 1 days before or when the fire started in atleast one grid
    global fire_occurred_dict
    missed_fire = ""
    fire_count = 0
    fire_occurred_dict = {}
    for i,row in df[df['fire_occurred']==1].iterrows():
        firee = row['fire_name'].lower()
        if firee not in fire_occurred_dict.keys():
            fire_occurred_dict[firee] = row['date']

    for fire in fire_occurred_dict.keys():
        if fire_occurred_dict[fire].month==1 and fire_occurred_dict[fire].day<11: #We didn't predict the first 10 days
            (fire_occurred_dict[fire])
        else:
            warn_date = fire_occurred_dict[fire] - timedelta(days=2)
            filter = (df['date']>warn_date) & (df['date']<=fire_occurred_dict[fire])
            #display(df[filter])
            if 1 not in df[filter]['predicted_fire_risk'].values:
                missed_fire = missed_fire + str(fire) + ","
            else:
                fire_count += 1
            
    temp = pd.DataFrame([[year,F1_score,F2_score,Precision,Recall,MCC,missed_fire,fire_count]],columns=cols)
    sam = sam.append(temp)
display(sam)
sam.to_csv('eval_metrics.csv',index=False)

C:\Users\sathishm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no true samples.

C:\Users\sathishm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no true samples.

C:\Users\sathishm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 due to no true samples.

C:\Users\sathishm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no true samples.

C:\Users\sathishm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to

,Year,F1_score,F2_score,Precision,Recall,MCC,missed_fire,Warned Fire Count
0,1999,0.00000,0.00000,0.00%,0.00%,0.00000,,0
0,2000,0.00000,0.00000,0.00%,0.00%,0.00000,,0
0,2001,0.00000,0.00000,0.00%,0.00%,-0.00250,"bell,",2
0,2002,0.00215,0.00536,0.11%,68.00%,0.02303,"gavilan,nate,",16
0,2003,0.01138,0.02792,0.57%,89.66%,0.06788,,14
0,2004,0.00107,0.00266,0.05%,54.72%,0.01300,"emerald,el moro,",27
0,2005,0.00147,0.00367,0.07%,78.95%,0.02039,,36
0,2006,0.00192,0.00478,0.10%,86.30%,0.02470,"gunn 2,",19
0,2007,0.03107,0.07373,1.58%,87.28%,0.10290,,28
0,2008,0.00072,0.00179,0.04%,85.71%,0.01496,,11


In [33]:
#df.to_parquet("LSTM_predictions",compression='gzip')
df = pq.read_table(source='LSTM_predictions').to_pandas()

In [34]:
df.sort_values(by=['latitude','longitude'], inplace=True,ascending=False)
df_copy = df[['latitude','longitude','predicted_prob_fire','fire_occurred']]

In [35]:
df_copy.head()

,latitude,longitude,predicted_prob_fire,fire_occurred
324485,33.566667,-116.058333,0.0,0
324486,33.566667,-116.058333,0.0,0
324487,33.566667,-116.058333,0.0,0
324488,33.566667,-116.058333,0.0,0
324489,33.566667,-116.058333,0.0,0


In [36]:
import plotly.figure_factory as ff
import matplotlib
import datetime
#matplotlib.use('TkAgg')
%matplotlib inline

In [37]:
sam = df[(df.date.dt.year==1999) & (df.date.dt.month==1) & (df.date.dt.day==1)]
lats = sorted(set(sam.latitude.values),reverse=True)
lons = sorted(set(sam.longitude.values))
lons = lons[6:47]
rect = np.zeros(shape =(len(lats),len(lons)))
rect_fire = np.empty(shape =(len(lats),len(lons)),dtype=str)
rect_mask = np.empty(shape =(len(lats),len(lons)), dtype=np.bool)
sns.set(font_scale=1.5)

In [38]:
def retrieve(year,month,day):
    sam = df[(df.date.dt.year==year) & (df.date.dt.month==month) & (df.date.dt.day==day)]
    temp = datetime.date(year=year, month=month, day=day)
    for i,lat in enumerate(lats):
        for j,lon in enumerate(lons):
            mask = (sam['latitude']==lat) & (sam['longitude']==lon)
            if sam[mask].size > 0:
                rect[i,j] =  (sam[mask]['predicted_prob_fire'].values[0])
                rect_mask[i,j] = False
                if int(sam[mask]['fire_occurred'])==1:
                    rect_fire[i,j]="F"
                else:
                    rect_fire[i,j]= ""
            else:
                rect_fire[i,j] = ""
                rect_mask[i,j] = True
#     fig1= px.imshow(rect,color_continuous_scale='Inferno',range_color=[0,1])
#     fig = ff.create_annotated_heatmap(rect,colorscale='Inferno',annotation_text=rect_fire)
#     fig.show()
#     fig1.show()
    fig2,ax = plt.subplots(figsize=(15,10))         # Sample figsize in inches
    sns.heatmap(rect, annot = rect_fire, mask = rect_mask, fmt = '',cmap='inferno',ax=ax,vmin = 0.0, vmax=1.0,yticklabels=False,xticklabels=False)
    Title = "Fire Potential over SD on " + str(temp)
    ax.set_title(Title)
    return()

In [48]:
interact_manual(retrieve,year=fixed(2017),month=[1,2,3,4,5,6,7,8,9,10,11,12],day=(1,30));

interactive(children=(Dropdown(description='month', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=1),…

In [ ]:
def retrieve_mean(year,month):
    sam = df[(df.date.dt.year==year) & (df.date.dt.month==month)]
    for i,lat in enumerate(lats):
        for j,lon in enumerate(lons):
            mask = (sam['latitude']==lat) & (sam['longitude']==lon)
            if sam[mask].size > 0:
                rect[i,j] =  (sam[mask]['predicted_prob_fire'].mean())
                rect_mask[i,j] = False
            else:
                rect_mask[i,j] = True
    fig2,ax = plt.subplots(figsize=(15,10))         # Sample figsize in inches
    sns.heatmap(rect,mask = rect_mask, fmt = '',cmap='inferno',ax=ax,vmin = 0.0, vmax=1.0,yticklabels=False,xticklabels=False)
    Title = "Fire Potential over SD on " + str(year) + " " + str(month)
    ax.set_title(Title)
    return()

In [ ]:
interact_manual(retrieve_mean,year=fixed(2010),month=[1,2,3,4,5,6,7,8,9,10,11,12]);

In [52]:
df[(df['fire_occurred']==1) & (df['date'].dt.month==3)]

,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,...,latitude,month,day,fire_occurred,acres_burned,fire_name,date,predicted_fire_risk,predicted_prob_nofire,predicted_prob_fire
279650,0.0,30.6,0.00633,186.2,247.0,3.5,295.0,283.5,38.0,10.7,...,33.483333,3,2,1,68.191520,GRANDE IC,2009-03-02,0.0,0.992340,0.007660
263955,0.0,32.0,0.00627,186.4,247.0,3.5,295.8,283.6,37.0,11.5,...,33.441667,3,2,1,68.191520,GRANDE IC,2009-03-02,0.0,0.989538,0.010462
208707,0.0,25.1,0.00554,277.4,54.0,4.3,299.3,281.7,32.0,16.6,...,33.275000,3,28,1,8.136972,COLE GRADE,2004-03-28,0.0,0.907300,0.092700
81096,0.0,17.1,0.00352,247.2,133.0,6.6,292.6,282.7,56.0,12.6,...,32.816667,3,8,1,270.074768,GUN,2014-03-08,0.0,1.000000,0.000000
81106,0.0,13.7,0.00290,252.7,317.0,6.4,292.1,276.0,74.0,6.3,...,32.816667,3,18,1,270.080000,Gun,2014-03-18,0.0,0.509859,0.490141
69781,0.0,18.9,0.00377,246.9,133.0,6.4,292.5,284.2,52.0,13.4,...,32.775000,3,8,1,270.074768,GUN,2014-03-08,0.0,1.000000,0.000000
69791,0.0,14.9,0.00311,252.1,317.0,6.2,292.2,276.3,70.0,6.6,...,32.775000,3,18,1,270.080000,Gun,2014-03-18,0.0,0.514943,0.485057
28168,0.0,28.2,0.00307,212.6,242.0,1.5,288.4,273.3,32.0,12.8,...,32.608333,3,5,1,6.982904,GUNN 2,2006-03-05,0.0,0.993386,0.006614
27803,0.0,28.3,0.00320,214.2,242.0,1.4,288.8,273.1,31.0,13.3,...,32.608333,3,5,1,6.982904,GUNN 2,2006-03-05,0.0,0.992217,0.007783
